## Attention 신경망 구현 및 학습 - need remind ! (Part 4 Lec 58-60)

In [1]:
import random
import tensorflow as tf
from konlpy.tag import Okt

## 데이터셋 준비


In [7]:
NUM_WORDS = 2000
MAX_LEN = 64     # 문장 길이

In [8]:
dataset_file = 'dataset/chatbot_data.csv' # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]   # morphs : 형태소 분석

questions = seq[::2]                               # 질문 : 홀수행
answers = ['\t ' + lines for lines in seq[1::2]]   # 답변 : 짝수행. tab = sos.

num_sample = len(questions)
num_sample

500

In [9]:
perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:  # train : 4/5
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)
        
print(len(train_q))
print(len(test_q))

399
101


In [10]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',  # !!!
                                                        maxlen=65)       # sos, eos 포함된 길이

x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

## Encoder

In [11]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, MAX_LEN)
        self.lstm = tf.keras.layers.LSTM(512, return_state=True)

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        _, h, c = self.lstm(x)   # hidden state, cell state
        return h, c

## Decoder

In [12]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, MAX_LEN)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        x, h, c = inputs
        x = self.emb(x)
        x, h, c = self.lstm(x, initial_state=[h, c])   # encoder에서 받은 것을 initial_state로 넣어줌
        
        return self.dense(x), h, c    # test 모드를 위해 h, c 리턴

## Seq2seq

In [13]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos       # start of sequence
        self.eos = eos       # end

    def call(self, inputs, training=False, mask=None):
        if training is True:
            x, y = inputs                 # y : shifted output
            
            print('x:', x.shape)
            print(x[0])
            print('y:', y.shape)
            print(y[0])
            
            h, c = self.enc(x)            # hidden state, cell state

            print('h:', h.shape)
            print('c:', c.shape)

            y, _, _ = self.dec((y, h, c)) 
            
            print('y__:', y.shape)
            print()
            
            return y
        
        else:               # test - 한 문장씩 테스트.
            x = inputs
            h, c = self.enc(x)
            
            # 첫번째 입력은 sos
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, MAX_LEN)

            for idx in tf.range(MAX_LEN):
                y, h, c = self.dec([y, h, c])  # 이전 스텝의 y, h, c를 입력으로 받음
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, MAX_LEN))

## 학습, 테스트 루프 정의

In [14]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]   # 학습시 입력으로 사용
    
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(output_labels, predictions)

@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [15]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [16]:
EPOCHS = 1

for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    if (epoch + 1)%10 == 0:
        template = 'Epoch {}, Loss: {:.5f}, Accuracy: {:.5f}'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              train_accuracy.result()))

    train_loss.reset_states()
    train_accuracy.reset_states()

x: (32, 64)
y: (32, 64)
h: (32, 512)
c: (32, 512)
y__: (32, 64, 2000)
x: (32, 64)
y: (32, 64)
h: (32, 512)
c: (32, 512)
y__: (32, 64, 2000)
x: (15, 64)
y: (15, 64)
h: (15, 512)
c: (15, 512)
y__: (15, 64, 2000)


## 테스트 루프

In [11]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q: ', test_text)
    print('a: ', gt_text)
    print('p: ', texts)

_
q:  ['여기 기프티콘 되죠 \n']
a:  ['\t 네 현금영수증 해드릴까 요 \n']
p:  ['여기 서 드실 건가 요 \n']
_
q:  ['네 에 테이크 아웃 도 가능한가요 \n']
a:  ['\t 네 로 오시 면 테이크 아웃 잔 에 담아 드려요 \n']
p:  ['네 고객 님 포인트 적립 하시겠어요 \n']
_
q:  ['아메리카노 톨 사이즈 로 주세요 \n']
a:  ['\t 따뜻한 거 로 드릴 까요 \n']
p:  ['다른 사항 은 없으신 가요 \n']
_
q:  ['진동 을 따로 주시나요 \n']
a:  ['\t 주 번호 로 드리겠습니다 \n']
p:  ['네 준비 해드리겠습니다 \n']
_
q:  ['자리 있나요 \n']
a:  ['\t 네 있습니다 \n']
p:  ['네 일회용 컵 에 담아 드리겠습니다 \n']
_
q:  ['그럼 루이보스 밀크 티 하나 \n']
a:  ['\t 네 알겠습니다 \n']
p:  ['플랫 화이트 는 카페라테 보다 우유 양 이 좀 적게 들어갔어요 \n']
_
q:  ['다음 에 무료 로 하고 엔 도장 찍어주세요 \n']
a:  ['\t 네 \n']
p:  ['네 이리 주세요 \n']
_
q:  ['아메리카노 한 잔 에 얼마 죠 \n']
a:  ['\t 입니다 \n']
p:  ['5000원 입니다 \n']
_
q:  ['얼마나 \n']
a:  ['\t 바로 만들어 드릴게요 \n']
p:  ['네 고객 님 결제 완료 되었습니다 \n']
_
q:  ['카푸치노 는 로 주시 고 아메리카노 는 로 \n']
a:  ['\t 네 더 없으세요 \n']
p:  ['드시고 가시나요 \n']
_
q:  ['아메리카노 는 어떤 종류 가 있나요 \n']
a:  ['\t 디카 페인 과 기본 아메리카노 2 종류 있습니다 \n']
p:  ['네 저기 서 직접 가져오시면 됩니다 \n']
_
q:  ['카카오 페이 로 결제 가능한가요 \n']
a:  ['\t 네 가능합니다 \n']
p:  ['네 그럼 바로 준비 해드리겠습니다 \n']
_
q: